In [35]:
import pandas as pd

Form a graph with people as nodes with edges between them if they have starred in the same movie. Search for some correlation between how well connected people are and the kind of movies they have performed in / the type of role they have etc.
Once such hypotheses have been formed, test them in a similar graph in the imdb movie dataset to see if they hold there as well. 

Test the hypothesis that an uncommon combination of genres means a somewhat extreme box office collection - either low or high



Is benford's law scale invariant?

In [36]:
movie_df = pd.read_csv('data/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'freebase_movie_id', 'title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres'], index_col='freebase_movie_id')
movie_df = movie_df.reset_index()

In [37]:
# No duplicates of freebase_movie_id
movie_df.groupby('freebase_movie_id').agg(count = ('freebase_movie_id', 'size')).sort_values('count', ascending=False)

,count
freebase_movie_id,
/m/011_mj,1
/m/0bwg_bz,1
/m/0bwgk61,1
/m/0bwgjgq,1
/m/0bwghsn,1
...,...
/m/04n51dq,1
/m/04n5187,1
/m/04n514t,1


In [ ]:
col_names = ["wiki_movie_id", "freebase_movie_id", "movie_release_date", "character_name", "actor_date_of_birth", "Actor gender", "Actor height (in meters)", "Actor ethnicity (Freebase ID)", "Actor name", "Actor age at movie release", "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
character_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names=col_names, index_col="Freebase character ID")

# Remove characters without actors
character_df = character_df[character_df['Freebase actor ID'].notna()]

# Allow only one character per actor per movie.
# Motivation: When we look at if the actor won a Oscar we don't care about what role it won it as. 
character_df = character_df.groupby(['freebase_movie_id', 'Freebase actor ID']).first().reset_index()

# How many cases where the actors of the same name stars in the same movie or
temp = character_df.groupby(['freebase_movie_id', 'Actor name']).agg(count = ('Actor name', 'size')).reset_index()
movies_with_actors_of_same_name_df = temp[temp['count'] > 1]
print('Number of movies with actors of the same name:', movies_with_actors_of_same_name_df.shape[0])

# Only 311 we decide to drop these cases
character_df = character_df.merge(temp, on=['freebase_movie_id', 'Actor name'], how='inner')
dropped_character_df = character_df[character_df['count'] == 1]

temp = dropped_character_df.groupby(['freebase_movie_id', 'Actor name']).agg(count = ('Actor name', 'size')).reset_index()

assert temp[temp['count'] > 1].empty

character_df = dropped_character_df

Number of movies with actors of the same name: 311


In [81]:
movies_with_actors_of_same_name_df.index

Index([  2143,   4403,   8130,  11947,  14725,  15002,  18533,  19532,  19672,
        20753,
       ...
       425858, 432836, 435555, 435637, 436926, 438201, 438272, 440291, 440452,
       442073],
      dtype='int64', length=311)

In [83]:
character_df.loc[movies_with_actors_of_same_name_df.index, :]

,freebase_movie_id,Freebase actor ID,wiki_movie_id,movie_release_date,character_name,actor_date_of_birth,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID
2143,/m/0160gs,/m/06k743,165359,1940,None,1902-12-25,M,NaN,None,Barton MacLane,37.0,/m/04ht_ng
4403,/m/01_4h2,/m/01k76k,358367,1977-11-03,The Mayor,1913-02-25,M,NaN,None,Jim Backus,64.0,/m/0h6dlgk
8130,/m/01ly1t,/m/0cj5mpy,255898,1979-04-12,Grinner,None,M,NaN,None,John Farndale,NaN,/m/0cj5mpm
11947,/m/021zgd,/m/03j24t0,384045,2000,None,1965-03-23,M,NaN,None,Richard Grieco,34.0,/m/09l2qtt
14725,/m/026f0cy,/m/02xv8m,7815982,1995-06-24,None,1960-01-12,M,1.92,/m/0g8_vp,Oliver Platt,35.0,/m/02vb3fx
...,...,...,...,...,...,...,...,...,...,...,...,...
438201,/m/0kv1wb,/m/0krwhp,7251650,None,None,1891-12-25,M,NaN,None,Earle Fox,NaN,/m/03jqrc3
438272,/m/0kv24t,/m/0krxsd,7255525,1912-08-02,The Groom,1884-04-28,M,NaN,None,William Garwood,NaN,/m/02tbb2v
440291,/m/0kv6w6,/m/0j92hnj,7412245,1914-12-15,Emma Lyons,None,F,NaN,None,Gladys Kingsbury,NaN,/m/0j94k24
440452,/m/0kv766,/m/07s3d36,7424038,1967,None,1913-06-23,M,NaN,None,Nicholas Phipps,53.0,/m/0gcl7q1


In [52]:
movie_character_df = movie_df.merge(character_df, on='freebase_movie_id', how='inner')
movie_character_df.columns

Index(['freebase_movie_id', 'wiki_movie_id_x', 'title', 'release_date',
       'box_office_revenue', 'runtime', 'languages', 'countries', 'genres',
       'Actor name', 'wiki_movie_id_y', 'movie_release_date', 'character_name',
       'actor_date_of_birth', 'Actor gender', 'Actor height (in meters)',
       'Actor ethnicity (Freebase ID)', 'Actor age at movie release',
       'Freebase character/actor map ID', 'Freebase actor ID'],
      dtype='object')

In [ ]:
# The same actor name can star in movies that are the same. 
# TODO: How do we handle this?
temp = movie_character_df.groupby(['Actor name', 'title']).agg(count = ('Actor name', 'size'))
temp.sort_values('count', ascending=False)

,,count
Actor name,title,
Bobby Deol,Barsaat,3
Mithun Chakraborty,Guru,3
The Doll Family,The Unholy Three,2
Lloyd Bridges,Little Big Horn,2
Sadha,Jayam,2
...,...,...
Gustavo Berriel,Olga,1
Gustavo Balbuena,Amor de otoño,1
Gustavo Angarita,The Day Silence Died,1


In [ ]:
# Duplicates in the dataset
movie_character_df[
    (movie_character_df['Actor name'] == 'Olivia Thirlby') & 
    (movie_character_df['title'] == 'The Other Side') & 
    (movie_character_df['character_name']== 'Max Mackenzie')]

,freebase_movie_id,wiki_movie_id_x,title,release_date,box_office_revenue,runtime,languages,countries,genres,wiki_movie_id_y,movie_release_date,character_name,actor_date_of_birth,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase actor ID
30260,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0h5qn1c,/m/025yjfc
30271,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0chbfb2,/m/025yjfc
30273,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0h38m4x,/m/025yjfc


In [ ]:
# Solve duplicates by allowing one actor per movie
# TODO: Explain very good why we do this for the TA Solve duplicates by allowing one actor per movie

In [18]:
movie_character_df[
    (movie_character_df['Actor name'] == 'Olivia Thirlby') & 
    (movie_character_df['title'] == 'The Other Side') & 
    (movie_character_df['character_name']== 'Max Mackenzie')]['countries']

30260    {"/m/09c7w0": "United States of America", "/m/...
30271    {"/m/09c7w0": "United States of America", "/m/...
30273    {"/m/09c7w0": "United States of America", "/m/...
Name: countries, dtype: object

In [56]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
oscar_df = oscar_df[oscar_df['film'].notna() & oscar_df['name'].notna()]

In [57]:
# Filtering relevant Oscar catagories
# TODO: explain why we drop certain categories.
oscar_df = oscar_df[oscar_df['category'].str.contains('ACTOR') | oscar_df['category'].str.contains('ACTRESS')]

This join needs to be checked. 
Joining with ignore case? 
Do we want to do a right join? (so also keep movies that did not win oscars)


In [58]:
# Avoid case-sensitivity and extra spaces
oscar_df['film_title_parsed'] = oscar_df['film'].str.lower().str.strip()
oscar_df['name_parsed'] = oscar_df['name'].str.lower().str.strip()

movie_character_df['film_title_parsed'] = movie_character_df['title'].str.lower().str.strip()
movie_character_df['name_parsed'] = movie_character_df['Actor name'].str.lower().str.strip()

In [59]:
oscar_character_movie_df = movie_character_df.merge(oscar_df, on=['film_title_parsed', 'name_parsed'], how='inner')